In [ ]:
import sys;
sys.path.insert(0, '..')

In [ ]:
from math import cos, floor, pi, sqrt, asin, sin

from util import generate_state, print_state_table, is_close, inner

from sim_circuit import *

## Exercise 1

Using the code introduced in this section, create a random state with $n = 4$ qubits and apply the classical magnitude amplification procedure for good outcomes 3 and 10.

**Answer:**

In [ ]:
def oracle(state, predicate):
    for item in range(len(state)):
        if predicate(item):
            state[item] *= -1

def inversion(original, current):
    proj = inner(original, current)
    for k in range(len(current)):
        current[k] = 2*proj*original[k] - current[k]

def grover_sim(state, predicate, iterations):
    s = state.copy()

    p = sum([abs(s[k])**2 for k in items])
    theta = asin(sqrt(p))
    assert is_close(inner(s, state), 1)

    for it in range(1, iterations + 1):
        oracle(state, predicate)
        inversion(s, state)
        assert is_close(inner(s, state), cos(2 * it * theta))

        p = sum([abs(state[k])**2 for k in items])
        assert is_close(p, sin((2 * it + 1)*theta)**2) 

In [ ]:
n = 4
items = [3, 11]
predicate = lambda i: True if i in items else False

state = generate_state(n)
print_state_table(state)

In [ ]:
num_iterations = int(floor(pi/4*sqrt(2**n/len(items))))

grover_sim(state, predicate, iterations = 2)

In [ ]:
print_state_table(state)

## Exercise 2

Create a magnitude amplification circuit for $n = 3$ qubits and single good outcome 5 using the circuit returned by the function from chapter 4 below as the initial state preparation operator (with `theta = 4*pi/7`).

In [ ]:
def prepare_binomial(n, theta):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for i in range(len(q)):
        qc.ry(theta, q[i])

    return qc

**Answer:**

In [ ]:
def is_bit_not_set(m, k):
    return not (m & (1 << k))

def phase_oracle_match(n, items):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for m in items:
        for i in range(n):
            if is_bit_not_set(m, i):
                qc.x(q[i])

        qc.mcp(pi, [q[i] for i in range(len(q) - 1)], q[len(q) - 1])

        for i in range(n):
            if is_bit_not_set(m, i):
                qc.x(q[i])
    return qc

In [ ]:
def inversion_0_circuit(n):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for i in range(n):
        qc.x(q[i])

    qc.mcp(pi, [q[i] for i in range(n - 1)], q[n - 1])

    for i in range(n):
        qc.x(q[i])

    return qc

In [ ]:
def inversion_circuit(A):
    n = sum(A.regs)
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    qc.append(A.inverse(), q)

    qc.append(inversion_0_circuit(n), q)

    qc.append(A, q)

    return qc

In [ ]:
def grover_iterate_circuit(A, O):
    n = sum(O.regs)
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    qc.append(O, q)

    qc.append(inversion_circuit(A), q)

    return qc

In [ ]:
def grover_circuit(A, O, iterations):
    n = sum(A.regs)
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    qc.append(A, q)

    for i in range(1, iterations + 1):
        qc.append(grover_iterate_circuit(A, O), q)
        qc.report(f'iteration_{i}')

    return qc

In [ ]:
n = 3
items = [5]
num_iterations = int(floor(pi/4*sqrt(2**n/len(items))))

theta = 4*pi/7
qc = grover_circuit(prepare_binomial(n, theta), phase_oracle_match(n, items), num_iterations)

In [ ]:
print_state_table(qc.run())